# Importing Libraries

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import re
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA


# Data Understanding and Preparation

## Reading Data

In [ ]:
artists_path = 'data\\artists.csv'
tracks_path = 'data\\tracks.csv'

This code automatically detects the correct separator for two dataset files (tracks and artists) by checking which character — comma, semicolon, or tab — appears most in the first line. It then loads each file into a pandas DataFrame using the detected separator, prints their shapes, and displays the first few rows.

 The tracks dataset has 11,166 rows and 45 columns, while the artists dataset has 104 rows and 14 columns.

In [ ]:
# Funzione helper per capire il separatore corretto
def detect_separator(filepath):
    with open(filepath, 'r', encoding='utf-8') as f:
        sample = f.readline()
    # Conta quanti separatori compaiono
    seps = {',': sample.count(','), ';': sample.count(';'), '\t': sample.count('\t')}
    best_sep = max(seps, key=seps.get)
    print(f"Detected separator for {filepath}: '{best_sep}'")
    return best_sep

# Rileva automaticamente il separatore
sep_tracks = detect_separator(tracks_path)
sep_artists = detect_separator(artists_path)

print('------------------------------------')

# Carica i dataset in base al separatore rilevato
tracks = pd.read_csv(tracks_path, sep=sep_tracks, encoding='utf-8', engine='python')
artists = pd.read_csv(artists_path, sep=sep_artists, encoding='utf-8', engine='python')

# Mostra alcune info per verifica
print(f"Tracks shape: {tracks.shape}")
print(f"Artists shape: {artists.shape}")
print('------------------------------------')

print('TRACKS')
display(tracks.head(3))

print('------------------------------------')
print('ARTISTS')
display(artists.head(3))


In [ ]:
print("Artists Features")
print(artists.columns.tolist())

print("Tracks Features")
print(tracks.columns.tolist())


## Duplicates

### Artists

The following code checks the artists dataset for duplicates in two ways: first, it looks for identical full rows to detect any completely repeated entries; then, it checks for duplicates specifically based on the artist ID and artist name columns.
<B> After performing both checks, it confirms that there are no duplicate artists in the dataset </B>.

In [ ]:
# Check for duplicated artists rows
duplicates_artists = artists[artists.duplicated()]

print(f"Number of duplicated Artists rows: {duplicates_artists.shape[0]}")
display(duplicates_artists.head(5))

In [ ]:
# Check for duplicated artists based on artist id
duplicates_artists_id = artists[artists.duplicated(subset='id_author')]
print(f"Number of duplicated artist based on ID: {duplicates_artists_id.shape[0]}")
display(duplicates_artists_id.head(5))



In [ ]:
# Check for duplicated artists based on artist name	
duplicates_artists_name = artists[artists.duplicated(subset='name')]
print(f"Number of duplicated artist based on Name: {duplicates_artists_name.shape[0]}")
display(duplicates_artists_name.head(5))

### Tracks
Duplicates rows check has been also performed here.
No duplicated rows were detected, indicating that all track entries are unique.

In [ ]:
# Check for duplicated tracks rows
duplicates_tracks = tracks[tracks.duplicated()]

print(f"Number of duplicated rows: {duplicates_tracks.shape[0]}")
display(duplicates_tracks.head(5))

#### Duplicated Tracks based on ID

This code checks the tracks dataset for duplicates based specifically on the track ID column. It identifies all rows where the same ID appears more than once, counts them, and displays them.
 It first identifies all rows where the same ID appears more than once, counts how many duplicated tracks exist, and displays them. Then, it counts how many times each track ID occurs in the dataset. 

<B> The result shows that there are 73 duplicated rows based on track IDs. 
Precisely we have 71  distinct IDs that have duplicates. </B>

<B>one track ID is repeated four times, while the others are each repeated twice </B>

In [ ]:
# Check for duplicated tracks based on track id
duplicates_tracks_id = tracks[tracks.duplicated(subset='id')]
print(f"Number of duplicated Tracks rows based on ID: {duplicates_tracks_id.shape[0]}")
display(duplicates_tracks_id)


# Count how many times each id_track appears
id_counts = tracks['id'].value_counts()
duplicate_id_counts = id_counts[id_counts > 1]

print('Number of distinct IDs that have duplicates')
print(duplicate_id_counts.size)
print("Number of tracks for each id:")
print(duplicate_id_counts)


The following code lists every full_title associated with each duplicated track ID. The results show 71 duplicated IDs in total. Most of these IDs are linked to two different songs, except for one ID that is associated with four songs (two pairs sharing the same title).

In [ ]:
# Find all duplicated track IDs
duplicate_ids = tracks[tracks.duplicated(subset='id', keep=False)]

# Group by 'id' and list all titles
titles_per_id = duplicate_ids.groupby('id')['full_title'].apply(list)

# Display each ID with all titles and the count of unique titles
for track_id, titles in titles_per_id.items():
    unique_count = len(set(titles))  # number of unique titles
    print(f"Track ID: {track_id} Number(of total songs: {len(titles)})(Unique titles: {unique_count})")
    for title in titles:
        print(f"  - {title}")
    print('----------------------------------------------------------')


#### Fixing Duplicated Tracks Id
After reviewing the songs associated with the duplicated IDs, we found that each duplicated ID corresponds to different songs, except for one case that will be treated later. Therefore, the most reasonable solution is to modify the duplicated IDs by appending the row number to each one. This approach ensures that all songs are preserved while maintaining unique identifiers for every track.

In [ ]:
# Identify duplicated IDs
duplicate_mask = tracks.duplicated(subset='id', keep=False)

# Assign new unique IDs only to duplicated rows
tracks.loc[duplicate_mask, 'id'] = (
    tracks.loc[duplicate_mask]
    .apply(lambda x: f"{x['id']}_{x.name}", axis=1)
)


print("Example of updated duplicates:")
display(tracks[duplicate_mask][['id', 'full_title']])


#### Duplicated Tracks based on Title
The following code identifies tracks that share the same full_title, meaning duplicate song titles. We found four duplicated tracks, corresponding to two pairs of songs with identical titles.

In [ ]:
# Find duplicated full_title
duplicate_titles = tracks[tracks.duplicated(subset='full_title', keep=False)]

# Sort by full_title to see them together
duplicate_titles = duplicate_titles.sort_values('full_title')

print(f"Tracks with duplicate track based on full_title: {duplicate_titles.shape[0]}")
display(duplicate_titles)


#### Fixing Duplicated Tracks full_title

The duplicated titles  — "BUGIE by Madame (Ft. Carl Brave & Rkomi)" and "sentimi by Madame" — actually refer to the same songs released in two different formats: one from the album and one from the single version. 
We decided to keep the duplicated tracks in the dataset but add a clear indication in the full_title to show whether each song comes from a single or an album. This way, all versions are preserved while making it easy to distinguish between different releases of the same song

In [ ]:

# Find duplicated full_titles
duplicate_mask = tracks.duplicated(subset='full_title', keep=False)

# Update only the duplicated titles by appending album_type
tracks.loc[duplicate_mask, 'full_title'] = (
    tracks.loc[duplicate_mask, 'full_title'] + 
    " (" + tracks.loc[duplicate_mask, 'album_type'].fillna('unknown').str.capitalize() + ")"
)

# Verify the changes
duplicate_titles = tracks[tracks.duplicated(subset='full_title', keep=False)].sort_values('full_title')
display(duplicate_titles[['full_title', 'album_type', 'id']])


## Merging the Datasets


Merging the tracks and artists datasets into a single DataFrame called df. It matches rows where the <B> id_artist column in tracks</B> corresponds to the <B>id_author column in artists</B>, using a left join so that all tracks are kept even if some artists are missing. After merging, it prints the number of rows and columns in the unified dataset and shows the first three rows for inspection.

In [ ]:
df = tracks.merge(artists, left_on='id_artist', right_on='id_author', how='left')

print(f"Unified dataset: {df.shape[0]} rows , {df.shape[1]} columns")
display(df.head(3))



Checking if there is a track without an artist

In [ ]:
# Check for tracks without a matching artist
missing_artists = df[df['id_author'].isna()]

print(f"Number of tracks without an artist: {missing_artists.shape[0]}")
display(missing_artists.head(5))

Checking if there is an artist without a track

In [ ]:
missing_artists = artists[~artists['id_author'].isin(tracks['id_artist'])]
print("Number of artists without any tracks:", len(missing_artists))

## Missing Values
This code analyzes missing values in the DataFrame by counting how many entries are NaN for each column and calculating the corresponding percentage. It creates a summary table showing only columns with missing data, sorted by the highest percentage.

In [ ]:
# Calcolo missing values e percentuali
missing_count = df.isna().sum()
missing_percent = (missing_count / len(df)) * 100

missing_df = (
    pd.DataFrame({'missing_count': missing_count, 'missing_percent': missing_percent})
    .sort_values('missing_percent', ascending=False)
    .query('missing_percent > 0')
)

# Mostra tabella riepilogativa (gradiente rosso-magenta)
display(
    missing_df
    .style.background_gradient(subset=['missing_percent'], cmap='RdPu')  
    .format({'missing_percent': '{:.2f}%'})
)


The following heatmap visualizes missing values in the dataset, with each row representing a record and each column a feature. Colored cells indicate missing entries, providing a clear overview of where data is incomplete.

In [ ]:
plt.figure(figsize=(14, 6))
sns.heatmap(df.isna(), cbar=False, cmap="viridis", yticklabels=False)
plt.title("Missing Values Matrix (Overview)", fontsize=20, pad=12, color="#000000")
plt.show()



The following bar plot shows the percentage of missing values per feature, with the top 20 features that have the most missing data

In [ ]:
plt.figure(figsize=(10, 8))
sns.barplot(
    data=missing_df.head(20),
    x='missing_percent',
    y=missing_df.head(20).index,
    hue=missing_df.head(20).index,  
    palette='RdPu_r'  
)
plt.title("Percentage of Missing Values by Feature", fontsize=20, pad=15, color="#000000")
plt.xlabel("Missing values (%)", fontsize=12)
plt.ylabel("Feature name", fontsize=12)

# Etichette percentuali
for index, value in enumerate(missing_df.head(20)['missing_percent']):
    plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#b30059')

plt.xlim(0, 100)
sns.despine()
plt.tight_layout()
plt.show()

#### Missing Values Propagation After Merge

In [ ]:
artists_missing = artists.isna().mean().sort_values(ascending=False) * 100
print(artists_missing)

The visualization highlights that missing values in attributes such as active_start, region, and birth_place have increased after merging due to the replication of incomplete artist metadata across multiple tracks.
This confirms that the merge process did not introduce new nulls, but propagated pre-existing ones.

In [ ]:
# Colonne provenienti dal dataset artists 
artist_cols =list(artists.columns)

# Conta i NaN prima e dopo il merge
missing_before = artists[artist_cols].isna().sum()
missing_after = df[artist_cols].isna().sum()

# Differenza assoluta e percentuale
missing_diff = missing_after - missing_before
increase_percent = (missing_diff / missing_before.replace(0, pd.NA)) * 100

# Tabella riepilogativa
missing_summary = (
    pd.DataFrame({
        "missing_before": missing_before,
        "missing_after": missing_after,
        "difference": missing_diff,
        "increase_%": increase_percent
    })
    .sort_values("difference", ascending=False)
)

plot_df = missing_summary[missing_summary['difference'] > 0].copy()

plt.figure(figsize=(10, 6))
sns.barplot(
    data=plot_df,
    x='difference',
    y=plot_df.index,
    hue=plot_df.index,
    palette='RdPu_r'
)
plt.title("Increase in Missing Values After Merge", fontsize=15, pad=12, color="#000000")
plt.xlabel("Increase in number of missing values", fontsize=12)
plt.ylabel("Feature", fontsize=12)

# Etichette numeriche a fianco delle barre
for index, value in enumerate(plot_df['difference']):
    plt.text(value + 50, index, f"{int(value):,}", va='center', fontsize=9, color="#000000")

sns.despine()
plt.tight_layout()
plt.show()

After analyzing the percentage of missing values in each column, We need to better understand the overall data quality before applying any filling strategies. Cleaning and validating the data first ensures that missing values are handled correctly and that no incorrect or misleading information is introduced during imputation.

## Data Quality

In [ ]:
df.info()

### Initial Data Cleaning

Based on the initial exploration of the dataset, we:

- **Removed empty column (`active_end`)** since it contained no useful information.  
- **Converted `popularity` and `year`** to numeric types to ensure consistency and enable statistical analysis.  
- **Transformed date-related columns** (`album_release_date`, `birth_date`, `active_start`, ) into proper datetime format for easier time-based operations.

Before directly converting year and popularity from objects to numeric and album_release_date, birth_date, and active_start from objects to datetime, we need to inspect the data to check if all values can be converted correctly and handle those that cannot be converted


In [ ]:
# 1. Remove empty column
df.drop(columns=['active_end'], inplace=True)  # drop the 'active_end' column because it's empty
df.info()

#### Objects to Numeric
Inspecting the values in popularity and year columns to see the values that cannot be converted to numbers directly

In [ ]:
numeric_cols = ['popularity','year']


# --- Check numeric columns ---
for col in numeric_cols:
    original = df[col].copy()
    converted = pd.to_numeric(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to numeric:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to numeric.")
    print('----------------------------------------------------------------')



Looking at the values of the  `popularity` column, we noticed that some entries contained **non-numeric characters**, percent signs (`%`), abbreviations like `K` (thousands) or `M` (millions), and words such as `"views"` appended to the numbers.  

Instead of converting the column directly to numeric using pd.to_numeric(errors='coerce'), which would have turned all invalid entries into NaN, we applied a cleaning function to preserve and correctly interpret useful numeric information before conversion. The function:

- Removed non-numeric characters and words like `"views"` and `%`.
- Converted abbreviations (`K → 1,000`, `M → 1,000,000`) to numeric values.
- Extracted the first numeric part if extra text was present.
- Converted the cleaned values to floats, marking any remaining invalid entries as `NaN`.


In [ ]:
def clean_popularity(value):
    if pd.isna(value):
        return None
    value_str = str(value).strip().lower()  # normalize
    
    # Remove common words like 'views'
    value_str = value_str.replace('views','').replace('%','').strip()
    value_str = value_str.lower()  
    # Handle K and M
    multiplier = 1
    if value_str.endswith('k'):
        multiplier = 1_000
        value_str = value_str[:-1]
    elif value_str.endswith('m'):
        multiplier = 1_000_000
        value_str = value_str[:-1]
    
    # Take only first token if words remain
    value_str = value_str.split()[0]
    
    # Try converting to float
    try:
        return (float(value_str) * multiplier)
    except:
        return None  # invalid entries become None/NaN
    
df['popularity'].apply(clean_popularity)




Inspecting the values in the year column, we observed that while most entries were numerical, some contained unexpected or non-numeric characters. To handle this, we converted the column directly to a numeric type using pd.to_numeric() with the errors='coerce' parameter, which automatically transforms any invalid or non-numeric values into NaN.

In [ ]:
df['year'] = pd.to_numeric(df['year'], errors='coerce') 

#### Objects to DateTime

Inspecting the values in 'album_release_date', 'birth_date', 'active_start' columns to see the values that cannot be converted to DateTime directly

In [ ]:
date_cols = ['album_release_date', 'birth_date', 'active_start']
# --- Check date columns ---
for col in date_cols:
    original = df[col].copy()
    converted = pd.to_datetime(original, errors='coerce')
    non_convertible = original[original.notna() & converted.isna()]
    
    print(f"\nColumn '{col}'  entries that cannot be converted to datetime:")
    if not non_convertible.empty:
        for idx, val in non_convertible.items():
            print(f"Row {idx}: {val}")
    else:
        print("All non-missing entries can be converted to datetime.")
    print('----------------------------------------------------------------')

Looking at the values in the album_release_date column that could not be converted to datetime, we noticed that many of them were just years (e.g., "2004"). If we used pd.to_datetime(errors='coerce') directly, these entries would have been turned into NaT. However, we wanted to keep this information by assigning a default month and day — the first day of the year.

- Instead of converting the column directly, we applied a cleaning function that:

- Detected values that were only a year (e.g., "2004") and changed them to a full date ("2004-01-01").

- Kept valid full dates (e.g., "2021-04-09") unchanged.

- Left missing values as they are.

- Finally, converted everything into proper datetime format for consistency.

In [ ]:
def fix_year_only_dates(val):
    """
    If the value looks like a 4-digit year, convert it to 'YYYY-01-01'.
    Otherwise, return the original value.
    """
    if pd.isna(val):
        return val
    val_str = str(val).strip()
    if re.fullmatch(r'\d{4}', val_str):
        return f"{val_str}-01-01"
    return val_str

# Apply to album_release_date
df['album_release_date'] = df['album_release_date'].apply(fix_year_only_dates)

# Convert to datetime
df['album_release_date'] = pd.to_datetime(df['album_release_date'], errors='coerce')

df.info()

Based on the values that could not be converted to datetime, we found that the birth_date column contained several invalid entries, such as URLs (e.g., "http://www.wikidata.org/.well-known/genid/...") instead of actual dates. Since these values do not represent meaningful or recoverable information, there is nothing worth preserving. Therefore, we are going to apply the pd.to_datetime(errors='coerce') function directly, allowing all invalid entries to be converted to NaT.

For the active_start column, all non-missing entries are  already in a valid date format, so they are going to be  successfully converted to datetime without any issues.

In [ ]:


date_cols = ['birth_date', 'active_start', ]
for col in date_cols:
    df[col] = pd.to_datetime(df[col], errors='coerce')  # convert to datetime, invalid dates become NaT


df.info()



### Data Distribution
The following table and histogram show the numerical data distribution in the dataset:

- **Most features** (`n_sentences`, `n_tokens`, `tokens_per_sent`, `char_per_tok`, `lexical_density`, `avg_token_per_clause`, `centroid`, `rolloff`, `rms`, `zcr`, `flatness`, `flux`, `spectral_complexity`, `pitch`, `loudness`) show **bell-shaped or near-normal distributions**.

- **Highly skewed features** (`stats_pageviews`, `bpm`, `tokens_per_sent`, `duration_ms`, `popularity`) have a **long right tail**, indicating a few extreme values or outliers (common in popularity or count-based features).

- **Temporal features** (`year`, `month`, `day`) display **non-uniform distributions**; e.g., `year` is concentrated around recent decades, showing most songs are modern.

- **Geographical features** (`latitude`, `longitude`) have **peaks corresponding to specific locations**, likely representing where artists or tracks are clustered.


In [ ]:

# Select numeric columns
num_cols = df.select_dtypes(include=['float64', 'int64']).columns

# --- Summary statistics table ---
display(df[num_cols].describe().T.style.background_gradient(cmap='RdPu'))

# --- Histograms for each numeric column ---
n_cols = 4
n_rows = -(-len(num_cols) // n_cols)  # ceil division
fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, n_rows * 4))
axes = axes.flatten()

for i, col in enumerate(num_cols):
    sns.histplot(df[col].dropna(), bins=30, kde=True, color="#d36ba8", ax=axes[i])
    axes[i].set_title(col, fontsize=18, color="#b30059")   # larger title font
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")
    axes[i].tick_params(axis='both', labelsize=12)          # larger tick labels

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("Distribution of Numerical Features", fontsize=24, color="#000000", y=1.02)  # larger main title
plt.tight_layout()
plt.show()



The data distribution and the statistics presented above reveal some anomalies and irregularities in the dataset. These issues will be examined and addressed in the following section.


###  Features Inspection Anomalies Detection

#### Track Year and Album release date
Looking at the distribution of values in the track year in the previous section, we notice some entries before 1950 and after 2025, which don’t make much sense. Similarly, there are album release dates after 2025 that seem unrealistic. Therefore, we will investigate these cases further to understand the cause and decide how to correct them.

The following code groups songs and albums into 20-year intervals based on their release years and visualizes the percentage distribution in each range. It first cleans and converts the year fields, then calculates how many songs or albums fall into each 20-year period.

Result:
From the plots, we can see that more than half of the songs and albums were released between 2000 and 2020, indicating that most of the data comes from the recent two decades

In [ ]:
# --- Convert 'year' to numeric ---
df['year'] = pd.to_numeric(df['year'], errors='coerce')

# --- Drop missing years and make a copy ---
df_years = df.dropna(subset=['year']).copy()

# --- Create 20-year bins ensuring last bin includes the max year ---
start = int(df_years['year'].min())
end = int(df_years['year'].max())
bins = list(range(start, end, 20)) + [end]  # ensure last bin ends exactly at max
labels = [f"{b}-{min(b+19, end)}" for b in bins[:-1]]

df_years['year_group'] = pd.cut(df_years['year'], bins=bins, labels=labels, right=False)

# --- Calculate percentage per group ---
group_percent = df_years['year_group'].value_counts(normalize=True).sort_index() * 100
group_df = pd.DataFrame({'year_group': group_percent.index, 'percent': group_percent.values})

# --- Plot ---
plt.figure(figsize=(10, 6))
sns.barplot(data=group_df, x='year_group', y='percent', hue='year_group', palette='viridis', legend=False)

# --- Add percentage labels ---
for i, val in enumerate(group_df['percent']):
    plt.text(i, val + 0.5, f"{val:.1f}%", ha='center', fontsize=10)

plt.title("Percentage of Songs by 20-Year Intervals", fontsize=18, pad=15)
plt.xlabel("Year Range", fontsize=12)
plt.ylabel("Percentage (%)", fontsize=12)
sns.despine()
plt.tight_layout()
plt.show()


# --- Convert 'album_release_date' to datetime and extract year ---
df['album_release_date'] = pd.to_datetime(df['album_release_date'], errors='coerce')
df['album_year'] = df['album_release_date'].dt.year

# --- Drop missing album years and make a copy ---
df_album_years = df.dropna(subset=['album_year']).copy()

# --- Create 20-year bins ensuring last bin includes the max year ---
start = int(df_album_years['album_year'].min())
end = int(df_album_years['album_year'].max())
bins = list(range(start, end, 20)) + [end]
labels = [f"{b}-{min(b+19, end)}" for b in bins[:-1]]

df_album_years['album_year_group'] = pd.cut(df_album_years['album_year'], bins=bins, labels=labels, right=False)

# --- Calculate percentage per group ---
group_percent = df_album_years['album_year_group'].value_counts(normalize=True).sort_index() * 100
group_df = pd.DataFrame({'album_year_group': group_percent.index, 'percent': group_percent.values})

# --- Plot ---
plt.figure(figsize=(10, 6))
sns.barplot(data=group_df, x='album_year_group', y='percent', hue='album_year_group', palette='mako', legend=False)

for i, val in enumerate(group_df['percent']):
    plt.text(i, val + 0.5, f"{val:.1f}%", ha='center', fontsize=10)

plt.title("Percentage of Songs by Album Release Year (20-Year Intervals)", fontsize=18, pad=15)
plt.xlabel("Album Release Year Range", fontsize=12)
plt.ylabel("Percentage (%)", fontsize=12)
sns.despine()
plt.tight_layout()
plt.show()


Descriptive Statistics
The summary statistics show that the song release years range from 1900 to 2100, with an average around 2013, indicating some unrealistic future values.
For album release years, the range is 1962 to 2025, with an average around 2017, which is more reasonable and reflects that most albums were released in the last decade.

In [ ]:
# For the 'year' column
print(df['year'].describe())  

# For 'album_release_date' (datetime type)
df['album_release_year'] = df['album_release_date'].dt.year
print(df['album_release_year'].describe())

Number of Songs before 1950 and after 2025 

In [ ]:
tracks['year'] = pd.to_numeric(tracks['year'], errors='coerce')

# Count songs released before 1950 and after 2025
songs_before_1950 = tracks[tracks['year'] < 1950].shape[0]
songs_after_2025 = tracks[tracks['year'] > 2025].shape[0]

print(f"Number of songs before 1950: {songs_before_1950}")
# Filter songs with year > 2025
future_songs = tracks[tracks['year'] <1950 ][['full_title', 'album_release_date', 'year']]
# Display the results
display(future_songs)

print(f"Number of songs after 2025: {songs_after_2025}")
# Filter songs with year > 2025
future_songs = tracks[tracks['year'] > 2025][['full_title', 'album_release_date', 'year']]
# Display the results
display(future_songs)


In [ ]:
tracks['album_release_date'] = pd.to_datetime(tracks['album_release_date'], errors='coerce')

cutoff_after = pd.to_datetime("2025-01-01")

album_release_date_after_2025 = tracks[tracks['album_release_date'] > cutoff_after].shape[0]

print(f"Number of album_release_date after 2025: {album_release_date_after_2025}")


#### Artist's BirthDate

In [ ]:
df['birth_year'] = df['birth_date'].dt.year

# --- Drop missing birth years and make a copy ---
df_birth_years = df.dropna(subset=['birth_year']).copy()

# --- Create 20-year bins ensuring last bin includes the max year ---
start = int(df_birth_years['birth_year'].min())
end = int(df_birth_years['birth_year'].max())
bins = list(range(start, end, 10)) + [end]
labels = [f"{b}-{min(b+9, end)}" for b in bins[:-1]]

df_birth_years['birth_year_group'] = pd.cut(df_birth_years['birth_year'], bins=bins, labels=labels, right=False)

# --- Calculate percentage per group ---
group_percent = df_birth_years['birth_year_group'].value_counts(normalize=True).sort_index() * 100
group_df = pd.DataFrame({'birth_year_group': group_percent.index, 'percent': group_percent.values})

# --- Plot ---
plt.figure(figsize=(10, 6))
sns.barplot(data=group_df, x='birth_year_group', y='percent', hue='birth_year_group', palette='coolwarm', legend=False)

# --- Add percentage labels ---
for i, val in enumerate(group_df['percent']):
    plt.text(i, val + 0.5, f"{val:.1f}%", ha='center', fontsize=10)

plt.title("Percentage of Artists by Birth Year (10-Year Intervals)", fontsize=18, pad=15)
plt.xlabel("Birth Year Range", fontsize=12)
plt.ylabel("Percentage (%)", fontsize=12)
sns.despine()
plt.tight_layout()
plt.show()


#### Active start

In [ ]:
df['active_start_year'] = df['active_start'].dt.year

# --- Drop missing active_start years and make a copy ---
df_active_years = df.dropna(subset=['active_start_year']).copy()

# --- Create 10-year bins ensuring last bin includes the max year ---
start = int(df_active_years['active_start_year'].min())
end = int(df_active_years['active_start_year'].max())
bins = list(range(start, end, 10)) + [end]
labels = [f"{b}-{min(b+9, end)}" for b in bins[:-1]]

df_active_years['active_year_group'] = pd.cut(df_active_years['active_start_year'], bins=bins, labels=labels, right=False)

# --- Calculate percentage per group ---
group_percent = df_active_years['active_year_group'].value_counts(normalize=True).sort_index() * 100
group_df = pd.DataFrame({'active_year_group': group_percent.index, 'percent': group_percent.values})

# --- Plot ---
plt.figure(figsize=(10, 6))
sns.barplot(data=group_df, x='active_year_group', y='percent', hue='active_year_group', palette='coolwarm', legend=False)

# --- Add percentage labels ---
for i, val in enumerate(group_df['percent']):
    plt.text(i, val + 0.5, f"{val:.1f}%", ha='center', fontsize=10)

plt.title("Percentage of Artists by Active Start Year (10-Year Intervals)", fontsize=18, pad=15)
plt.xlabel("Active Start Year Range", fontsize=12)
plt.ylabel("Percentage (%)", fontsize=12)
sns.despine()
plt.tight_layout()
plt.show()


#### Popularity

In [ ]:
# --- Count popularity values ---
pop_counts = (df['popularity'].astype(str)).value_counts().sort_index()  # sort index for ascending y-axis

# --- Horizontal bar plot ---
plt.figure(figsize=(10, 20))
sns.barplot(x=pop_counts.values, y=pop_counts.index,hue=pop_counts.index, palette='viridis')
plt.xlabel("Number of Songs", fontsize=12)
plt.ylabel("Popularity", fontsize=12)
plt.title("Distribution of Song Popularity", fontsize=16, pad=15)

# --- Add count labels ---
for i, val in enumerate(pop_counts.values):
    plt.text(val + 0.5, i, f"{val}", va='center', fontsize=9)

plt.tight_layout()
plt.show()


####  Artists Location Statistics


##### Checking if all the coordinates of the artists are inside italy's coordinates

In [ ]:
geo_outliers = df[(df['latitude'] < 35.5) | (df['latitude'] > 47.1) |
                  (df['longitude'] < 6.6) | (df['longitude'] > 18.5)]
print(f"Number of Geographic coordinates outside Italy range: {len(geo_outliers)} records")
display(geo_outliers[['name_artist', 'latitude', 'longitude', 'birth_place']].head(10))

##### Artists' Country Values

All the countries have the value of Italia

In [ ]:
# Count the occurrences of each country
country_counts = df['country'].value_counts()

# Plot
plt.figure(figsize=(10, 6))
country_counts.plot(kind='bar', color='skyblue', edgecolor='black')

plt.title('Distribution of Artists by Country', fontsize=14, pad=12)
plt.xlabel('Country', fontsize=12)
plt.ylabel('Number of Artists', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()


##### Checking if there is an artist his/her country not Italy but his/her coordinates are in Italy

In [ ]:
# Filter rows where country is not Italy and coordinates are present
non_italy_with_coords = df[
    (df['country'].notna()) & 
    (df['country'] != "Italia") & 
    (df['latitude'].notna()) & 
    (df['longitude'].notna())
]

# Count the number of such records
num_records = len(non_italy_with_coords)
print(f"Number of non-Italy records with coordinates: {num_records}")

# Show the records
print(non_italy_with_coords[['country', 'latitude', 'longitude']])


##### Artists Nationality Distribution

Almost all artists are Italian (99.5%), with a small minority from Argentina (0.5%).

In [ ]:

print(df['nationality'].value_counts())
# Count and calculate percentages
nat_counts = df['nationality'].value_counts()
nat_percent = (nat_counts / nat_counts.sum()) * 100
nat_df = nat_percent.reset_index()
nat_df.columns = ['nationality', 'percent']

# Plot
plt.figure(figsize=(10, 8))
sns.barplot(
    data=nat_df.head(20),  # show top 20 nationalities
    x='percent',
    y='nationality',
    hue='nationality',
    palette='crest',
    dodge=False
)

plt.title("Percentage of Artists by Nationality", fontsize=18, pad=15)
plt.xlabel("Percentage (%)", fontsize=12)
plt.ylabel("Nationality", fontsize=12)

# Add percentage labels
for index, value in enumerate(nat_df.head(20)['percent']):
    plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

plt.xlim(0, nat_df['percent'].max() + 5)
sns.despine()
plt.tight_layout()
plt.show()


##### Checking if there are artists with Non-Italian Nationality and Italian Coordinates (doubt)

There are 40 artists with a nationality other than Italian (all Argentinian) but also have italian geographic coordinates. All these 40 artists share the same coordinates (After searching for this coordinates refers to the province of Parma).

In [ ]:
# Filter rows where country is not Italy and coordinates are present
non_italy_with_coords = df[
    (df['nationality'].notna()) & 
    (df['nationality'] != "Italia") & 
    (df['latitude'].notna()) & 
    (df['longitude'].notna())
]

# Count the number of such records
num_records = len(non_italy_with_coords)
print(f"Number of non-Italy Nationality records with coordinates: {num_records}")

# Show the records
print(non_italy_with_coords[['nationality','latitude', 'longitude']])


##### Nationality and Country Coherence Check  (doubt)

This check ensures that each artist’s nationality matches the country. For example, artists from Italy should have nationality Italia, and those from Argentine should have Argentina.
The results show no mismatches, meaning all records have consistent country–nationality relationships.

In [ ]:

# Example mapping of country → expected nationality
country_to_nationality = {
    "Italy": "Italia",
    "Argentine": "Argentina",
}


# Function to check nationality vs country
def check_nationality_country(row):
    if pd.notna(row['country']) and pd.notna(row['nationality']):
        expected_nationality = country_to_nationality.get(row['country'])
        if expected_nationality and expected_nationality != row['nationality']:
            return True  # incoherent
    return False  # coherent or missing data

# Apply the function
df['nationality_country_mismatch'] = df.apply(check_nationality_country, axis=1)

# Count mismatches
num_mismatches = df['nationality_country_mismatch'].sum()
print(f"Number of nationality-country mismatches: {num_mismatches}")

# Show records with mismatch
mismatched_records = df[df['nationality_country_mismatch']]
print(mismatched_records[['country', 'nationality']])


##### Distribution of Artist's Birth Places

The majority of artists were born in major Italian cities, with Milano (1,843) and Roma (1,048) being the most frequent birthplaces, indicating a strong concentration of artists from these cultural and economic centers.

Smaller Italian towns such as Senigallia (443), Torino (397), and Avellino (329) also show notable representation, suggesting a widespread national distribution beyond just the biggest cities.

Only a few artists were born outside Italy — such as Buenos Aires (40) and Almería (26) — representing less than 1% of the total, which confirms that the dataset is predominantly composed of Italian-born artists.

In [ ]:
# Count occurrences and calculate percentages
birth_place_counts = df['birth_place'].value_counts()
print(birth_place_counts)
birth_place_percent = (birth_place_counts / len(df)) * 100

# Plot
plt.figure(figsize=(14, 6))
bars = plt.bar(birth_place_counts.index, birth_place_counts.values, color='skyblue')

# Labels and title
plt.title('Distribution of Birth Places', fontsize=14)
plt.xlabel('Birth Place')
plt.ylabel('Count')

# Add both count and percentage labels above bars
for i, (count, percent) in enumerate(zip(birth_place_counts.values, birth_place_percent.values)):
    plt.text(i, count + 10, f"{count:,} \n({percent:.1f}%)", 
             ha='center', va='bottom', fontsize=6, color='black')

plt.xticks(rotation=45, ha='right')  # Rotate labels for readability
plt.tight_layout()
plt.show()


##### Checking Birth Place–Country Consistency (doubt)

This section verifies whether each artist’s birth place matches their country. It defines a list of known Italian cities and maps a few foreign cities to their respective countries. T The result shows the number of mismatches and lists the inconsistent records. The results show 26 mismatches, all involving artists born in Almería (Spain) but recorded with the country Italia

In [ ]:

# List of Italian cities from the data
italian_cities = [
    "Milano", "Roma", "Senigallia", "Torino", "Avellino", "Cagliari", "Salerno",
    "Olbia", "Napoli", "Vimercate", "Vicenza", "Verona", "Scampia", "Nicosia",
    "Sternatia", "Padova", "Grottaglie", "La Spezia", "Scafati", "Nocera Inferiore",
    "Sesto San Giovanni", "Genova", "Alpignano", "Fiumicino", "Treviso", "Bologna",
    "San Siro", "Rho", "Brescia", "Grugliasco", "Reggio Calabria", "Gallarate",
    "Desenzano del Garda", "Pieve Emanuele", "San Benedetto del Tronto", "Firenze",
    "Lodi"
]

# Map known foreign cities to their countries
foreign_cities_to_country = {
    "Singapore": "Singapore",
    "Buenos Aires": "Argentina",
    "Almería": "Spagna",
}


# Function to check birth_place vs country
def check_birth_place_country(row):
    if pd.notna(row['birth_place']) and pd.notna(row['country']):
        if row['birth_place'] in italian_cities and row['country'] != "Italia":
            return True  # mismatch
        elif row['birth_place'] in foreign_cities_to_country:
            if row['country'] != foreign_cities_to_country[row['birth_place']]:
                return True  # mismatch
    return False  # coherent or missing data

# Apply the function
df['birth_place_country_mismatch'] = df.apply(check_birth_place_country, axis=1)

# Count mismatches
num_mismatches = df['birth_place_country_mismatch'].sum()
print(f"Number of birth_place-country mismatches: {num_mismatches}")

# Show records with mismatch
mismatched_records = df[df['birth_place_country_mismatch']]
display(mismatched_records[['birth_place', 'country',]])


##### Birth Place vs Nationality Consistency Check

This step verifies that each artist’s birth place aligns with their nationality. A list of Italian cities and a mapping of known foreign cities (like Almería, Buenos Aires, and Singapore) were used for comparison.

The results show 107 mismatches, mainly involving artists born in Almería or Singapore but labeled with the nationality Italia, indicating possible errors or inconsistencies in the dataset.

In [ ]:
# List of Italian cities
italian_cities = [
    "Milano", "Roma", "Senigallia", "Torino", "Avellino", "Cagliari", "Salerno",
    "Olbia", "Napoli", "Vimercate", "Vicenza", "Verona", "Scampia", "Nicosia",
    "Sternatia", "Padova", "Grottaglie", "La Spezia", "Scafati", "Nocera Inferiore",
    "Sesto San Giovanni", "Genova", "Alpignano", "Fiumicino", "Treviso", "Bologna",
    "San Siro", "Rho", "Brescia", "Grugliasco", "Reggio Calabria", "Gallarate",
    "Desenzano del Garda", "Pieve Emanuele", "San Benedetto del Tronto", "Firenze",
    "Lodi"
]


# Map special foreign cities to nationality
foreign_cities_to_nationality = {
    "Singapore": "Singapore",
    "Buenos Aires": "Argentina",
    "Almería": "Spagna",
}

# Function to check birth_place vs nationality
def check_birth_place_nationality(row):
    if pd.notna(row['birth_place']) and pd.notna(row['nationality']):
        if row['birth_place'] in italian_cities and row['nationality'] != "Italia":
            return True  # mismatch
        elif row['birth_place'] in foreign_cities_to_nationality:
            if row['nationality'] != foreign_cities_to_nationality[row['birth_place']]:
                return True  # mismatch
    return False  # coherent or missing data

# Apply the function
df['birth_place_nationality_mismatch'] = df.apply(check_birth_place_nationality, axis=1)

# Count mismatches
num_mismatches = df['birth_place_nationality_mismatch'].sum()
print(f"Number of birth_place-nationality mismatches: {num_mismatches}")

# Show records with mismatch
mismatched_records = df[df['birth_place_nationality_mismatch']]
print(mismatched_records[['name','birth_place', 'nationality','country']])


##### Distribution of Songs by Province and Region

This code calculates and visualizes the percentage distribution of songs by province and region. It counts occurrences, converts them to percentages, and displays bar charts with labeled values to show which areas have the highest song representation

In [ ]:

# Count occurrences and convert to percentages
province_counts = df['province'].value_counts()
province_percent = (province_counts / province_counts.sum()) * 100
print('Provinces')
print(province_counts)

# Create a DataFrame for plotting
province_df = province_percent.reset_index()
province_df.columns = ['province', 'percent']


plt.figure(figsize=(10, 8))
sns.barplot(
    data=province_df.head(20),  # top 20 provinces if you want
    x='percent',
    y='province',
    hue='province',
    palette='viridis',
    dodge=False
)

plt.title("Percentage of Songs by Province", fontsize=20, pad=15, color="#000000")
plt.xlabel("Percentage (%)", fontsize=12)
plt.ylabel("Province", fontsize=12)

# Add percentage labels
for index, value in enumerate(province_df.head(20)['percent']):
    plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

plt.xlim(0, province_df['percent'].max() + 5)
sns.despine()
plt.tight_layout()
plt.show()



region_counts = df['region'].value_counts()
print('Regions')
print(region_counts)
region_percent = (region_counts / region_counts.sum()) * 100
region_df = region_percent.reset_index()
region_df.columns = ['region', 'percent']

plt.figure(figsize=(10, 8))
sns.barplot(
    data=region_df,
    x='percent',
    y='region',
    hue='region',
    palette='coolwarm',
    dodge=False
)

plt.title("Percentage of Songs by Region", fontsize=20, pad=15, color="#000000")
plt.xlabel("Percentage (%)", fontsize=12)
plt.ylabel("Region", fontsize=12)

# Add percentage labels
for index, value in enumerate(region_df['percent']):
    plt.text(value + 0.5, index, f"{value:.1f}%", va='center', fontsize=9, color='#000000')

plt.xlim(0, region_df['percent'].max() + 5)
sns.despine()
plt.tight_layout()
plt.show()


##### Province/Region – Country Consistency Check

This code verifies that Italian provinces and regions are correctly associated with the country "Italia"

In [ ]:
# Example mapping of Italian regions to their provinces (from your data)
region_provinces = {
    "Lombardia": ["Milano", "Monza e della Brianza", "Brescia", "Varese", "Lodi"],
    "Campania": ["Salerno", "Napoli", "Avellino"],
    "Lazio": ["Roma"],
    "Veneto": ["Vicenza", "Verona", "Padova", "Treviso"],
    "Piemonte": ["Torino"],
    "Sardegna": ["Cagliari", "Gallura"],
    "Puglia": ["Lecce", "Taranto"],
    "Liguria": ["Genova", "La Spezia"],
    "Sicilia": ["Enna"],
    "Emilia-Romagna": ["Bologna"],
    "Calabria": ["Reggio Calabria"],
    "Marche": ["Ancona", "Ascoli Piceno"],
    "Toscana": ["Firenze"]
}

# Flatten all Italian provinces for quick lookup
all_italian_provinces = [prov for provs in region_provinces.values() for prov in provs]

# Function to check province/region ↔ country
def check_province_region_country(row):
    if pd.notna(row['country']):
        if pd.notna(row['province']) and row['province'] in all_italian_provinces:
            if row['country'] != "Italia":
                return True  # mismatch
        elif pd.notna(row['region']) and row['region'] in region_provinces.keys():
            if row['country'] != "Italia":
                return True  # mismatch
    return False  # coherent or missing data

# Apply the function
df['province_region_country_mismatch'] = df.apply(check_province_region_country, axis=1)

# Count mismatches
num_mismatches = df['province_region_country_mismatch'].sum()
print(f"Number of province/region-country mismatches: {num_mismatches}")

# Show records with mismatch
mismatched_records = df[df['province_region_country_mismatch']]
print(mismatched_records[['province', 'region', 'country']])


##### Province/Region vs Birth Place – Consistency Check (doubt)

This check compares each artist’s birth_place with the corresponding province and region. Mismatches occur when the province or region does not align with the birth_place. There are 2,901 mismatches between birth_place and province/region.

In [ ]:
# Updated mapping of Italian regions to provinces including all birth_places in your data
region_provinces = {
    "Lombardia": ["Milano", "Vimercate", "Sesto San Giovanni", "Alpignano", "Fiumicino",
                  "Brescia", "Grugliasco", "Rho", "Gallarate", "Desenzano del Garda", "Lodi", "San Siro"],
    "Lazio": ["Roma"],
    "Piemonte": ["Torino"],
    "Campania": ["Salerno", "Napoli", "Avellino", "Scafati", "Nocera Inferiore"],
    "Veneto": ["Vicenza", "Verona", "Padova", "Treviso"],
    "Sardegna": ["Cagliari", "Olbia", "Gallura"],
    "Puglia": ["Lecce", "Taranto", "Grottaglie", "Sternatia", "San Benedetto del Tronto"],
    "Liguria": ["Genova", "La Spezia"],
    "Sicilia": ["Enna", "Nicosia"],
    "Emilia-Romagna": ["Bologna"],
    "Calabria": ["Reggio Calabria"],
    "Marche": ["Ancona", "Senigallia", "Ascoli Piceno"],
    "Toscana": ["Firenze", "Scampia", "Padova"]
}

# Flatten province → region mapping
province_to_region = {prov: reg for reg, provs in region_provinces.items() for prov in provs}

# Function to check birth_place ↔ province/region
def check_birth_place_province_region(row):
    if pd.notna(row['birth_place']):
        # Only check Italian birth_places
        if row['birth_place'] in province_to_region:
            expected_region = province_to_region[row['birth_place']]
            # Compare province and region if available
            if (pd.notna(row['province']) and row['province'] != row['birth_place']) or \
               (pd.notna(row['region']) and row['region'] != expected_region):
                return True  # mismatch
    return False  # coherent or missing data

# Apply the function
df['birth_place_province_region_mismatch'] = df.apply(check_birth_place_province_region, axis=1)

# Count mismatches
num_mismatches = df['birth_place_province_region_mismatch'].sum()
print(f"Number of birth_place-province/region mismatches: {num_mismatches}")

# Show mismatched records
mismatched_records = df[df['birth_place_province_region_mismatch']]
print(mismatched_records[['birth_place', 'province', 'region']])


##### Geographic Distribution of Artists by Province and Region

This analysis aggregates the number of artists by their latitude, longitude, province, and region. The resulting table shows the locations with the highest concentration of artists at the top. For example, Milano (Lombardia) has the most artists with 1,843, followed by Roma (Lazio) with 1,048, and Torino (Piemonte) with 397. The code also generates a map where the size and color of the points reflect the number of artists per location, providing a clear visual of artist density across Italy.

In [ ]:
# Aggregate by latitude and longitude to count number of artists
location_counts = df.groupby(['latitude', 'longitude', 'region', 'province']).size().reset_index(name='num_artists')

# Sort by number of artists descending
location_counts = location_counts.sort_values(by='num_artists', ascending=False)

# Print the sorted table
print(location_counts)

# Define a color scale
color_scale = [(0, 'orange'), (1,'red')]

# Create the scatter map
fig = px.scatter_mapbox(
    location_counts,
    lat="latitude",
    lon="longitude",
    hover_data=["region", "province", "num_artists"],  # show count on hover
    size="num_artists",  # size of marker represents number of artists
    color="num_artists",  # color also shows density
    color_continuous_scale=color_scale,
    zoom=5,
    height=800,
    width=800
)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()


# Outliers Detection

In [ ]:
df_clean = df.copy()
print(f"Original size: {df_clean.shape}")

# Handle critical NaNs
df_clean = df_clean.dropna(subset=['lyrics'])
print(f"Rows with missing 'lyrics' removed.")
print(f"Size after handling NaNs: {df_clean.shape}")

In [ ]:
# Numerical Feature Definition

# List of key numerical columns to analyze
numerical_features = [
    'stats_pageviews', 'swear_IT', 'swear_EN',
    'n_sentences', 'n_tokens', 'tokens_per_sent', 'char_per_tok', 'lexical_density', 'avg_token_per_clause',
    'bpm', 'centroid', 'rolloff', 'flux', 'rms', 'zcr', 'flatness', 'spectral_complexity', 'pitch', 'loudness',
    'duration_ms', 'popularity'
]

# Ensure all are numeric and remove any rows with NaNs
df_clean[numerical_features] = df_clean[numerical_features].apply(pd.to_numeric, errors='coerce')
df_clean = df_clean.dropna(subset=numerical_features)
print(f"DataFrame ready for outlier analysis. Size: {df_clean.shape}")

In [ ]:
# Visual Analysis (Box Plot)

for col in numerical_features:
    plt.figure(figsize=(12, 3))
    sns.boxplot(data=df_clean, x=col, color="skyblue")
    plt.title(f"Box Plot of '{col}'")
    plt.grid(axis='x', linestyle='--', alpha=0.6)
    plt.show()

In [ ]:
# Statistical Analysis

outlier_report = []

for col in numerical_features:
    Q1 = df_clean[col].quantile(0.25)
    Q3 = df_clean[col].quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    count_low = df_clean[df_clean[col] < lower_bound].shape[0]
    count_high = df_clean[df_clean[col] > upper_bound].shape[0]
    total_count = df_clean[col].count()

    if total_count > 0:
        total_perc = (count_low + count_high) / total_count * 100
    else:
        total_perc = 0

    outlier_report.append({
        'feature': col,
        'lower_bound': round(lower_bound, 2),
        'upper_bound': round(upper_bound, 2),
        'outliers_low': count_low,
        'outliers_high': count_high,
        'total_outliers': count_low + count_high,
        'total_perc': round(total_perc, 2)
    })

# Print the report
outlier_df = pd.DataFrame(outlier_report).set_index('feature')
print(outlier_df.sort_values(by='total_perc', ascending=False))

In [ ]:
# Filtering and plotting outlier data
plot_data = outlier_df[outlier_df['total_outliers'] > 0]

# Sort the number of outliers in descending order
plot_data = plot_data.sort_values(by='total_outliers', ascending=False)
plot_data = plot_data[['outliers_low', 'outliers_high']]

print(f"Found {len(plot_data)} features with outliers.")

if plot_data.empty:
    print("No outliers to plot.")
else:

    colors = ['#d36ba8', '#b51272']

    ax = plot_data.plot(
        kind='barh',
        stacked=True,
        figsize=(12, 10),
        color=colors,
        width=0.8
    )

    plt.title('Outlier counts  (method 1.5 x IQR)', fontsize=16, pad=20)
    plt.xlabel('Number of Ouliers', fontsize=12) # 'Ouliers' è probabilmente un refuso per 'Outliers'
    plt.ylabel('Feature', fontsize=12)

    plt.gca().invert_yaxis()

    plt.legend(
        title='Type of Outlier',
        labels=['Low Outlier (< lower bound.)', 'High Outlier (> Upper bound.)'],
        loc='lower right'
    )

    plt.tight_layout()
    plt.show()

1. Skewed Features (Needing Log-Transform): The report shows that features like stats_pageviews (8.93% outliers) and swear_EN (8.85%) are not normally distributed; they are highly skewed. The box plots (e.g., image_ce5d57.png for stats_pageviews) visually confirm this, showing the data is "squashed" to one side with a long tail of outliers. These are perfect candidates for Strategy 1: Log Transformation.

2. Other Outliers (Needing Clipping): The rest of the features (like avg_token_per_clause, lexical_density, loudness, etc.) have a smaller, more manageable percentage of outliers (mostly 1-4%). These represent legitimate but extreme values. They are ideal for Strategy 2: Clipping, which will reduce their influence without deleting them.

## Gestione Outlier

In [ ]:
# Trasformazione Log

skewed_cols = [
    'stats_pageviews',
    'swear_EN',
    'avg_token_per_clause',
    'swear_IT',
    'rms',
    'duration_ms',
    'n_sentences',
    'n_tokens',
    'rolloff',
    'zcr'
]

for col in skewed_cols:
    if col in df_clean.columns:
        new_col_name = f"{col}_log"
        df_clean[new_col_name] = np.log1p(df_clean[col])
        print(f"'{new_col_name}' Column created.")


In [ ]:
# Visual Check of Transformation
for col in skewed_cols:
    col_log = f"{col}_log"

    if col in df_clean.columns and col_log in df_clean.columns:

        plt.figure(figsize=(14, 5))

        # --- Plot 1: Original Distribution ---
        plt.subplot(1, 2, 1)
        sns.histplot(df_clean[col], kde=True, bins=50, color='#d36ba8')
        plt.title(f'Original Distribution of \n{col}', fontsize=14)
        plt.xlabel('Original Value')
        plt.ylabel('Count')

        # --- Plot 2: Transformed Distribution ---
        plt.subplot(1, 2, 2)
        sns.histplot(df_clean[col_log], kde=True, bins=50, color='#d36ba8')
        plt.title(f'Transformed Distribution of \n{col_log}', fontsize=14) #
        plt.xlabel('Log-Transformed Value (log(1+x))')
        plt.ylabel('Count')


        plt.suptitle(f'Transformation Check for: {col}', fontsize=18, y=1.05)
        plt.tight_layout()
        plt.show()

In [ ]:
# Clipping

clipping_cols = [col for col in numerical_features if col not in skewed_cols]

for col in clipping_cols:
    if col in df_clean.columns:

        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR


        count_low = (df_clean[col] < lower_bound).sum()
        count_high = (df_clean[col] > upper_bound).sum()


        if (count_low + count_high) > 0:
            df_clean[col] = df_clean[col].clip(lower_bound, upper_bound)
            print(f"Clipping applied to '{col}': {count_low+count_high} values clipped.")

In [ ]:
# Final Check After Outlier Handling

transformed_cols = [f"{col}_log" for col in skewed_cols]

# Combine the lists for the final check
final_feature_list = transformed_cols + clipping_cols

In [ ]:
# Box plot

for col in final_feature_list:
    if col in df_clean.columns:
        plt.figure(figsize=(12, 3))
        # Usiamo 'color' per evitare il FutureWarning
        sns.boxplot(data=df_clean, x=col, color="skyblue")
        plt.title(f"Box Plot Finale di '{col}'")
        plt.grid(axis='x', linestyle='--', alpha=0.6)
        plt.show()

In [ ]:
# Statistical Check (Final IQR Report)

final_outlier_report = []

for col in final_feature_list:
    if col in df_clean.columns and pd.api.types.is_numeric_dtype(df_clean[col]):

        Q1 = df_clean[col].quantile(0.25)
        Q3 = df_clean[col].quantile(0.75)
        IQR = Q3 - Q1

        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        count_low = (df_clean[col] < lower_bound).sum()
        count_high = (df_clean[col] > upper_bound).sum()

        total_count = df_clean[col].count()

        if total_count > 0:
            total_perc = (count_low + count_high) / total_count * 100
        else:
            total_perc = 0

        final_outlier_report.append({
            'feature': col,
            'lower_bound': round(lower_bound, 2),
            'upper_bound': round(upper_bound, 2),
            'outliers_low': count_low,
            'outliers_high': count_high,
            'total_outliers': count_low + count_high,
            'total_perc': round(total_perc, 2)
        })

# Print the final report
if final_outlier_report:
    final_outlier_df = pd.DataFrame(final_outlier_report).set_index('feature')
    print(final_outlier_df.sort_values(by='total_perc', ascending=False))

In [ ]:
# Filtering and plotting outlier data
plot_data = final_outlier_df[final_outlier_df['total_outliers'] > 0]

# Sort the number of outliers in descending order
plot_data = plot_data.sort_values(by='total_outliers', ascending=False)
plot_data = plot_data[['outliers_low', 'outliers_high']]

print(f"Found {len(plot_data)} features with outliers.")

if plot_data.empty:
    print("No outliers to plot.")
else:

    colors = ['#d36ba8', '#b51272']

    ax = plot_data.plot(
        kind='barh',
        stacked=True,
        figsize=(12, 10),
        color=colors,
        width=0.8
    )

    plt.title('Outlier counts  (method 1.5 x IQR)', fontsize=16, pad=20)
    plt.xlabel('Number of Ouliers', fontsize=12)
    plt.ylabel('Feature', fontsize=12)

    plt.gca().invert_yaxis()

    plt.legend(
        title='Type of Outlier',
        labels=['Low Outlier (< lower bound.)', 'High Outlier (> Upper bound.)'],
        loc='lower right'
    )

    plt.tight_layout()
    plt.show()

1. "Zero-Inflated" Features (Statistical Artifact)
Most of the data is 0. This means Q1=0 and Q3=0, so the IQR is 0. The statistical rule Q3 + (1.5 * IQR) calculates an upper bound of 0.
The rule is "broken" for this data. It flags every non-zero value as an outlier, even after transformation.

2. "Natural Tails" of a New Bell-Curve
Example: n_sentences_log (711 outliers), n_tokens_log (634), stats_pageviews_log (17)
The log transform successfully changed these skewed distributions into a symmetrical, "bell-shaped" (normal) distribution, as seen in your histograms.
The 1.5 * IQR rule is very strict. The outliers it now finds are not errors; they are simply the natural tails of the new, healthy bell-curve.

### Multivariate analisys

In [ ]:
# Ensure there are no NaNs (should be clean already)
df_analysis = df_clean[final_feature_list].dropna()
print(f"Data ready for analysis: {df_analysis.shape}")

scaler = StandardScaler()
data_scaled = scaler.fit_transform(df_analysis)

In [ ]:
# Applying Isolation Forest
iso_forest = IsolationForest(contamination=0.02, random_state=42)

# We train and get the predictions
# The algorithm assigns:
#  1 for Inliers (normal points)
# -1 for Outliers (anomalous points)
predictions = iso_forest.fit_predict(data_scaled)

df_analysis['is_outlier_multi'] = predictions

df_clean['is_outlier_multi'] = df_analysis['is_outlier_multi'].reindex(df_clean.index)

outliers_multi = df_clean[df_clean['is_outlier_multi'] == -1]
print(f"\nAnalysis completed.")
print(f"Number of multivariate outliers identified: {len(outliers_multi)}")

# Show some of the records identified as anomalous
print("\nExamples of Multivariate Outliers:")

# Show the original columns and our clean columns
display(outliers_multi[['full_title', 'primary_artist'] + final_feature_list].head())

It identified the 93 (2%) songs that are the most stylistically unique when combining all 21 features.
The examples show two clear patterns:

Lyrical Anomalies (Rosa Chemical): This artist is flagged repeatedly. The data shows his songs have a very rare combination: they are lyrically complex (avg_token_per_clause_log, lexical_density) AND have high profanity in both Italian and English (swear_IT_log, swear_EN_log). This makes them stand out from all other artists.

Audio Anomalies (thasup): The "thasup" track is a perfect example of an audio outlier. It has a very slow bpm (82) but is at the maximum loudness (45) and maximum pitch (3191). This combination of "slow, loud, and high-pitched" is a very unusual audio profile.

Conclusion: These 93 songs are "stylistic outliers" (like experimental tracks). They are not errors, but you should remove them before clustering to get clearer, more representative clusters of the main "rap schools".

In [ ]:
# We use PCA to reduce the dimension and be able to plot the outliers

# Reduce the scaled data to 2 dimensions
pca = PCA(n_components=2, random_state=42)
data_scaled_2d = pca.fit_transform(data_scaled)

# Create a DataFrame for plotting
plot_df = pd.DataFrame(data_scaled_2d, columns=['PC1', 'PC2'])
plot_df['is_outlier'] = predictions

# Create the Scatter Plot
plt.figure(figsize=(12, 8))
sns.scatterplot(
    data=plot_df,
    x='PC1',
    y='PC2',
    hue='is_outlier',
    palette={1: 'blue', -1: 'red'}, # Outliers in red
    alpha=0.6
)
plt.title('Isolation Forest Results (visualized with PCA)')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.legend(title='Type', labels=['Outlier (-1)', 'Inlier (1)'])
plt.show()

# Correlation analysis

In [ ]:
# Select the clean data (only the inliers)
if 'df_clustering' not in locals():
    df_clustering = df_clean[df_clean['is_outlier_multi'] != -1]

df_corr = df_clustering[final_feature_list].copy()
print(f"Data ready for correlation analysis: {df_corr.shape}")

# Calculate the correlation matrix (Pearson Method)
corr_matrix = df_corr.corr(method='pearson')

# Visualize the Heatmap
plt.figure(figsize=(20, 16))

# Create a "mask" to hide the upper part
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(
    corr_matrix,
    mask=mask,
    annot=True,
    fmt='.1f',
    cmap='coolwarm',
    linewidths=0.5,
    cbar_kws={"shrink": .8}
)
plt.title('Feature Correlation Matrix', fontsize=20, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

It clearly shows strong multicollinearity (redundancy) among your features. The data isn't noisy; it just confirms that some features measure the same underlying concept.
- Song Length: duration_ms_log, n_tokens_log, and n_sentences_log are highly correlated (0.8 to 0.9), as they all measure "song length".

- Loudness: rms_log and loudness are almost identical (0.9), measuring "volume".

- Spectral Brightness: centroid, rolloff_log, and flatness are very strongly correlated (0.9 and -0.8), measuring the "brightness" or "timbre" of the sound.

In [ ]:
# 2. Select the clean data (BUT INCLUDING the multivariate outliers)
df_corr_completo = df_clean[final_feature_list].copy()

df_corr_completo = df_corr_completo.dropna()
print(f"Data ready for correlation analysis: {df_corr_completo.shape}")

# 3. Calculate the correlation matrix (Pearson Method)
corr_matrix_completo = df_corr_completo.corr(method='pearson')

# 4. Visualize the Heatmap
plt.figure(figsize=(20, 16))

mask = np.triu(np.ones_like(corr_matrix_completo, dtype=bool))

sns.heatmap(
    corr_matrix_completo,
    mask=mask,
    annot=True,
    fmt='.1f',
    cmap='coolwarm',
    linewidths=0.5,
    cbar_kws={"shrink": .8}
)
plt.title('Correlation Matrix (Including the 93 outliers from Isolation Forest)', fontsize=20, pad=20)
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()